# Gai/Gen: Text-to-Text (TTT)

## 1.1 Setting Up

We will create a seperate virtual environment for this to avoid conflicting dependencies that each underlying model requires.

```sh
sudo apt update -y && sudo apt install ffmpeg git git-lfs -y
conda create -n TTT python=3.10.10 -y
conda activate TTT
pip install gai-gen[TTT]
```

The following examples has been tested on the following environment:

-   NVidia GeForce RTX 2060 6GB
-   Windows 11 + WSL2
-   Ubuntu 22.04
-   Python 3.10
-   CUDA Toolkit 11.8
-   openai 1.6.1
-   anthropic 0.8.1
-   transformers 4.36.2
-   bitsandbytes 0.41.3.post2
-   scipy 1.11.4
-   accelerate 0.25.0
-   llama-cpp-python 0.2.25


## 1.2 Running as a Library

For (1) and (2) below, you will use the GaiGen library to call OpenAI's GPT4.
You will need to get an API key from OpenAI. 
Create .env file in project root directory and insert the OpenAI API Key below:

```sh
OPENAI_API_KEY=<your key here>
```

In [ ]:
### 1. GPT4 Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('gpt-4')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}])
print(response.choices[0].message.content)

In [ ]:
### 2. GPT4 Text-to-Text Streaming

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('gpt-4')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],stream=True)
for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content,end='',flush=True)

For (3) and (4), you will run Mistral 7B locally. Vlone TheBloke's 4-bit quantized version of Mistral-7B model from hugging face. 
This model utilizes the exLlama loader for increased performance.

```sh
git clone https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GPTQ ~/gai/models/Mistral-7B-Instruct-v0.1-GPTQ
```

In [1]:
### 3. Mistral Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b-exllama')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100)
print(response.choices[0].message.content,end='',flush=True)

GENERATING:
Once upon a time, in a small village surrounded by a dense forest, there lived a young girl named Lily. She was known for her kindness and love for animals. One day, while wandering through the woods, she stumbled upon an injured bird. She took it home and nursed it back to health. As days passed, the bird regained its strength and flew away. In return, the bird brought Lily many beautiful flowers from the forest. From then on, whenever she needed a friend

In [2]:
### 4. Mistral Text-to-Text Streaming

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b-exllama')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100,stream=True)
for chunk in response:
    print(chunk.choices[0].delta.content,end='',flush=True)

STREAMING:
 Once upon a time, in a small village nestled at the foot of a mountain, there lived a young girl named Emma. Despite being born into a poor family, Emma was kind-hearted and always helped those in need. One day, while collecting firewood in the forest, she stumbled upon an old woman who was lost and hungry. Emma took the old woman home, fed her, and gave her a place to stay. The next day, the old woman revealed that she was actually

The following example uses Anthropics Claude2.1 100k context window size model. Get API Key from Anthropics and add it to the .env file.
```sh
ANTHROPIC_APIKEY=<your key here>
```

In [ ]:
### 5. Claude-2.1 Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('claude2-100k')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_tokens_to_sample=100)
print(response.choices[0].message.content,end='',flush=True)

In [ ]:
### 6. Claude-2.1 Text-to-Text Streaming

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('claude2-100k')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_tokens_to_sample=100,stream=True)
for chunk in response:
    print(chunk.choices[0].delta.content,end='',flush=True)

Follow the instructions [here](https://huggingface.co/docs/transformers/main/en/model_doc/llama2) to signup with Meta to download the LLaMa-2 model.
Download the model in HuggingFace format from [here] (https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) into ~/gai/models/Llama-2-7b-chat-hf.

In [ ]:
### 7. Llama2-7B Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
from IPython.utils import io
with io.capture_output() as captured:
    gen = Gaigen.GetInstance().load('llama2-transformers')
    response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100)
print(response.choices[0].message.content,end='',flush=True)

In [ ]:
### 8. Llama2-7B Text-to-Text Streaming

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('llama2-transformers')
from IPython.utils import io
with io.capture_output() as captured:
    response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100,stream=True)
for chunk in response:
    print(chunk.choices[0].delta.content,end='',flush=True)

The following example uses GGUF formatted version of Mistral-7B for LlaMaCPP. This can be used when you want the model to run off CPU only.
Follow this instruction to download TheBloke's Mistral-7B GGUF model:
```
mkdir ~/gai/models/Mistral-7B-Instruct-v0.1-GGUF && cd $_
wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf
```

In [ ]:
## 9. Mistral-7B CPU-Only Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b-llamacpp')
from IPython.utils import io
with io.capture_output() as captured:
    response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100)
print(response.choices[0].message.content,end='',flush=True)

In [ ]:
## 10. Mistral-7B CPU-Only Text-to-Text Generation

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b-llamacpp')
from IPython.utils import io
with io.capture_output() as captured:
    response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100,stream=True)
for chunk in response:
    print(chunk.choices[0].delta.content,end='',flush=True)

## 1.3 Running as a Service

### Step 1: Start Docker container

```bash
docker run -d \
    --name gai-ttt \
    -p 12031:12031 \
    --gpus all \
    -v ~/gai/models:/app/models \
    kakkoii1337/gai-ttt:latest
```

### Step 2: Wait for model to load

```bash
docker logs gai-ttt
```

When the loading is completed, the logs should show this:

```bash
INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:12031 (Press CTRL+C to quit)
```

### Step 3: Test

In [ ]:
%%bash
#### Step 3: Test

curl -X POST \
    http://localhost:12031/gen/v1/chat/completions \
    -H 'Content-Type: application/json' \
    -s \
    -N \
    -d "{\"model\":\"mistral7b-exllama\", \
        \"messages\": [ \
            {\"role\": \"user\",\"content\": \"Tell me a story\"}, \
            {\"role\": \"assistant\",\"content\": \"\"} \
        ],\
        \"max_new_tokens\":25, \
        \"stream\":true}"
